# GRU
LSTM과 비슷한 역할을 하지만, 더 간단한 구조로 이뤄져 있어 계산상 효율적이다. 이것은 기존의 LSTM에서 사용되는 셀 상태 계산(은닉 상태 업데이트)을 줄였다. 또한, 특정 문제에서는 LSTM보다 더 적합한 레이어다.

## GRU 구조
- GRU는 LSTM에서의 셀 상태(cell state) 역할의 c가 없다. cell state의 역할을 다음의 출력 h에서 그 역할을 함께 한다.
- GRU에서는 Update Gate, Reset Gate 두 가지만 존재하며 사용되는 활성화 함수는 sigmoid 2번과 tanh 1번 사용된다.

- Reset Gate : 과거의 데이터를 리셋시키는 것을 목적으로 하는 게이트, sigmoid 연산을 통해 과거의 데이터를 얼마나 리셋시킬지에 대한 값인 r(0과 1사이)을 출력
- Update Gate : 과거와 현재의 정보 업데이트 비율을 결정하는 게이트, 출력값 u는 현시점에서의 가져가야 할 데이터 양을 결정하는 값. 1-u는 잊어버려야 할 데이터 양이라고 생각하면 된다.

- 데이터 선정
과거의 출력 값 데이터를 그대로 이용하지 않고, 리셋 데이터로 출력한 값을 이용하여 pointwise 곱 연산을 한다. 연산 결과는 tanh함수를 한번 거치게 된다.
- 출력값 계산
u는 데이터 중 얼마나 가져갈 것이지를, 1-u는 얼마나 잊을 것인지를 의미한다. tanh를 거친 h~는 현시점에서 리셋된 데이터를 의미하며 해당 데이터 중 가져갈 것을 연산한다. 이전의 h에서 얼마나 잊을지를 연산하여 이것들을 모두 합한 값이 출력값이 된다.

In [1]:
# test 환경
import numpy as np

X = []
Y = []
for i in range(6):
    lst = list(range(i,i+4))
    X.append(list(map(lambda c: [c/10], lst)))
    Y.append((i+4)/10)
X = np.array(X)
Y = np.array(Y)
print(X)
print(Y)

[[[0. ]
  [0.1]
  [0.2]
  [0.3]]

 [[0.1]
  [0.2]
  [0.3]
  [0.4]]

 [[0.2]
  [0.3]
  [0.4]
  [0.5]]

 [[0.3]
  [0.4]
  [0.5]
  [0.6]]

 [[0.4]
  [0.5]
  [0.6]
  [0.7]]

 [[0.5]
  [0.6]
  [0.7]
  [0.8]]]
[0.4 0.5 0.6 0.7 0.8 0.9]


In [2]:
# 모델 만들기 & 학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential([GRU(units=32,
              # units 파라미터는 RNN 신경망에 존재하는 뉴런의 개수
                    return_sequences=False,
        # RNN 계산 과정에 있는 hidden state를 출력할 것인지에 대한 값을 의미
        # 해당 값은 다층으로 이루어진 RNN 또는 one-to-many, many-to-many 출력을 위해서 사용
        # False의 경우, 마지막 출력값 하나를 출력/ True의 경우, 모든 과정의 출력 값을 출력
                    input_shape=[4,1]),
                    Dense(1, activation='tanh')])
# 모델은 Sequential에 simpleRNN과 Dense레이어를 하나씩 추가
# input_shape는 4 time-step마다 하나의 답이라서 [4,1] 선언
# 학습을 반복하면서 mse 값을 낮추는 훈련 함

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=200, verbose=0)

In [3]:
# 기존의 훈련 데이터 세트를 넣어서 예측
print(model.predict(X))

[[0.43649778]
 [0.5346203 ]
 [0.6234701 ]
 [0.70119697]
 [0.7669906 ]
 [0.82101744]]


In [4]:
# 1.2에 많이 가깝진 않지만 훈련데이터를 증가시켜서 해당 결과를 개선시킬 수 있음
# RNN 사용시 0.912, LSTM 사용시 0.936, GRU 사용시 0.923
X_test = np.array([[[0.8],[0.9],[1.0],[1.1]]])
print(model.predict(X_test))

[[0.9237259]]
